# Nucleus detection and sub image generation

I this notebook I (Dominik) tried out code with the aim to generate a pipeline that would generate the subimages. This pipeline was ultimately used to generate the subimages that we used for the manual labelling and training of classification models.

In [1]:
# import packages
import cv2
import numpy as np
import os
from datetime import datetime

The nucleus detection will be a very crucial step for the user to get a good quantification. If not all cells are detected or a lot of artifacts are picked up, the predictions will not be accurate. That is why I decided to put the important variables for the nucleus detect in an external field. Later in the app the user will be able to fine-tune these values in order to get the most optimal nucleus detecion.

In [2]:
# define the paths for the folders

nuclei_path = "../data/withoutmarker_newdata_split/blue/blue-green"
merged_images_folder = "../data/unlabeled images"
subimages_output_folder = "../data/subimages"

# if this boolean is set to True, you will follow the nuclei detection for each image one by one and have to push any button to continue
follow_nuclei_detection = False

# changeable variables that can be tweaked for the nuclei detection

# size of the kernel that is used to close holes in objects (if smaller the objects are more separated). inputing 5 will generate a 5x5 kernel
kernelsize = 5

# aspect ratio treshold, how much is the shape allowed to diverge from the ratio of 1, which would indicate that hight and width are identical (0.4 was good)
aspect = 1

# circularity treshold, it is 1 for a perfect circle. How much is the object allowed to diverge from that? (0.7 was good)
circ = 1

# decide for min and max area of the detected object
min_size = 300
max_size = 10000

In [3]:
# function to detect the nuclei
def detect_objects2(image_path, output_dir, kernelsize=kernelsize, aspect=aspect, circ=circ, min_size=min_size, max_size=max_size):
    '''
    Function detects nucleus center coordinates.
    image_path: give the path to the respective image
    output_dir: give the path where the nucleus detected images are supposed to be saved
    additional input can be the user input for defining the shape of the objects to be detected
    returns a list with the nucleus center coordinates
    '''
    # Load the image from the respective folder given above
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"Image file not found at the path: {image_path}")

    # Apply a Gaussian blur to the image
    blurred = cv2.GaussianBlur(image, (11, 11), 0)

    # Apply thresholding to create a binary image
    _, thresh = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY)

    # Apply morphological closing to fill small holes (this is sometimes necessary, because neutrophil nuclei can be segmented, which means the we may want to detect some smaller objects as one)
    kernel = np.ones((kernelsize, kernelsize), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Dilate the image to merge close contours
    dilated = cv2.dilate(closing, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw detected contours as red circles
    color_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    coordinates = []

    # now we iterate through each detected contour and decide, if it is an object we want
    for contour in contours:
        # Get the moments to calculate the center of the contour
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            
            # Calculate aspect ratio (draw a rectangular box around the contour, aspect ratio = width/height of that box) and circularity (is 1 for a perfect cycle)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * (area / (perimeter * perimeter))

            # Filter based on aspect ratio, circularity and min/max area, the aspect ratio should be close to 1 for round objects and the circularity of a perfect circle is 1
            if 1 - aspect <= aspect_ratio <= 1 + aspect and circularity > 1 - circ and min_size <= area <= max_size:
                coordinates.append((cX, cY))
                # Draw the center of the contour with a red dot
                cv2.circle(color_image, (cX, cY), 5, (0, 0, 255), -1)

   # Save the image with detected centers
    output_image_path = os.path.join(output_dir, f"{os.path.basename(image_path)}_detected.png")
    cv2.imwrite(output_image_path, color_image)

    # Display the image with detected centers, active by setting follow_nuclei_detection as True above
    if follow_nuclei_detection:
        cv2.imshow("Detected Centers", color_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return coordinates

In [6]:
# now we use these coordinates to do the subimaging on the merge-channel images

def extract_sub_images(image_path, coordinates, output_dir):
    ''' 
    Function extracts the subimages around the coordinates given.
    image_path: give the image path of the multi-channel images
    coordinates: list of nucleus center coordinates (from detect_objects2 function)
    output_dir: give path for output of the subimages
    returns a list of the subimages
    '''
    # Load the multi-channel image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image file not found at the path: {image_path}")

    # Make sure the given output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    sub_images = []
    for i, (cX, cY) in enumerate(coordinates):
        # Define the bounding box for the sub-image. I figured out that a 60 x 60 pixel box would be perfect. Thus, we subtract/add 30 to each side starting from the coordinates
        startX = max(cX - 30, 0)
        startY = max(cY - 30, 0)
        endX = min(cX + 30, image.shape[1])
        endY = min(cY + 30, image.shape[0])
        
        # Extract the sub-image from the respective multi-channel image
        sub_image = image[startY:endY, startX:endX]
        sub_images.append(sub_image)
        
        # Save the sub-image to the output directory
        sub_image_path = os.path.join(output_dir, f"{os.path.basename(image_path)}_sub_image_{i}.png")
        cv2.imwrite(sub_image_path, sub_image)

    return sub_images

## Batch process

Since this is working pretty well, I now want to write some functions to do the subimaging automatically for the whole folder

In [8]:
# this function uses the detect_objects2 function and iterates through the folder of the nuclei images defined in the beginning

def process_images_in_folder(folder_path, output_dir):
    ''' 
    Function uses detect_objects2 function and iterates through the folder of the nuclei images.
    folder_path: path of the base folder of the images
    output_dir: path where the images are saved
    returns a list of coordinate-lists
    '''
    # List to store coordinates of all images
    all_coordinates = {}
    
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp')):
            image_path = os.path.join(folder_path, filename)
            print(f"Processing {image_path}...")
            # getting the coordinates through the detect objects function
            coordinates = detect_objects2(image_path, output_dir)
            all_coordinates[filename] = coordinates
    
    # all_coordinates now is a list of the coordinate_lists
    return all_coordinates

In [11]:
# this function now iterates through all the multi-channel images to create the subimages around the coordinates saved above

def main(nuclei_folder, input_folder, output_folder_base):
    ''' 
    Function assembles all the functions above to generate the subimages
    nuclei_folder: path of the folder with the blue-channel images
    input_folder: path of the folder with the multi-channel images
    output_folder_base: path of the base folder for the output
    '''
    # Generate a timestamp for the output folder name
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    output_folder = os.path.join(output_folder_base, f"output_{timestamp}")

    # Create the main output folder and subfolders
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    nuclei_detection_output_dir = os.path.join(output_folder, "nucleus_detection")
    if not os.path.exists(nuclei_detection_output_dir):
        os.makedirs(nuclei_detection_output_dir)

    # Process all images in the folder to detect nuclei
    all_coordinates = process_images_in_folder(nuclei_folder, nuclei_detection_output_dir)

    # Extract and save sub-images for each image
    for filename, coordinates in all_coordinates.items():
        image_path = os.path.join(input_folder, filename)
        image_path = image_path[:-11]
        sub_image_output_dir = os.path.join(output_folder, 'subimages')
        print(f"Extracting sub-images for {image_path}...")
        extract_sub_images(image_path, coordinates, sub_image_output_dir)


In [12]:
main(nuclei_path, merged_images_folder, subimages_output_folder)

Processing ../data/withoutmarker_newdata_split/blue/blue-green/20211111.lif_PMA 3.11.21NET488_MPO633_006.tif (blue).jpg...
Processing ../data/withoutmarker_newdata_split/blue/blue-green/20211111.lif_ctr 3.11.21NET488_MPO633_014.tif (blue).jpg...
Processing ../data/withoutmarker_newdata_split/blue/blue-green/20211111.lif_PMA 3.11.21NET488_MPO633_009.tif (blue).jpg...
Processing ../data/withoutmarker_newdata_split/blue/blue-green/20211111.lif_PMA 3.11.21NET488_MPO633_012.tif (blue).jpg...
Processing ../data/withoutmarker_newdata_split/blue/blue-green/20211111.lif_ctr 3.11.21NET488_MPO633_017.tif (blue).jpg...
Processing ../data/withoutmarker_newdata_split/blue/blue-green/20211111.lif_PMA 3.11.21NET488_MPO633_003.tif (blue).jpg...
Processing ../data/withoutmarker_newdata_split/blue/blue-green/20211111.lif_ctr 3.11.21NET488_MPO633_011.tif (blue).jpg...
Processing ../data/withoutmarker_newdata_split/blue/blue-green/20211111.lif_ctr_27.10.21NET488_MPO 633_004.tif (blue).jpg...
Processing ../